In [1]:
import nltk
import sys 
import re
from sklearn.decomposition  import  PCA
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from nltk.corpus import stopwords

In [2]:
train=pd.read_json("./train.zip")
test=pd.read_json("./test_questions.zip")

In [3]:
train=train[["id","text"]]
test=test[["id","text"]]

In [4]:
train

,id,text
0,1016267329137729536,"@Fhumu_Makhwanya Hi, no you cannot withdraw fu..."
1,1016270646060384256,"@GTalevi Hi, can you please share more informa..."
2,1016272104839434240,"@Mandilele_ Hi, thank you for sharing your exp..."
3,1016275249753743360,@GTalevi Please DM us your Home-Loan account n...
4,1016277410545889281,@Angvanreenen83 Please provide us with it.
...,...,...
96557,1030149995490697217,"@Dopla101 Hi, apologies for the delayed respon..."
96558,1030162359933906945,"@BlackRose__ZA Hi.\nKindly DM your query, affe..."
96559,1030170340473929730,"@aminah_bhengu Hi, please inbox us your ID &am..."
96560,1030172343937716224,"@reitumakwea Hi, please follow &amp; inbox us ..."


In [5]:
test

,id,text
0,1028546931453243392,@_Warith Hi Wari. please contact our Customer ...
1,1028548116297986050,"@ziyaad_seedat Hi there Ziyaad, the Samsung Pa..."
2,1028550157699350528,@180DegreesCC Lovely packaging. How long have ...
3,1028552399307399169,"@ConsolerM Hi there, please give us your conta..."
4,1028559460745150465,@zimcritic Loving yourself looks beautiful on ...
...,...,...
32205,1032883399898423298,"@lee_masilo Hi @lee_masilo, sorry for late res..."
32206,1032886868902334464,@BeckeChris Thanks Chris. Network informed. Ne...
32207,1032887467526049792,@ntandoz_nmz You're welcome
32208,1032888433788825601,"@sne_mthethwa Hi sne_mthethwa, we would like t..."


In [24]:
train['text'] = train['text'].replace('amp', 'and', regex=True)
test['text'] = test['text'].replace('amp', 'and', regex=True)
train['text'] = train['text'].replace('_', '', regex=True)
test['text'] = test['text'].replace('_', '', regex=True)

train['text'] = train['text'].str.replace('[^\w\s]','')
test['text'] = test['text'].str.replace('[^\w\s]','')

test['text'] = test['text'].str.lower()
train['text'] = train['text'].str.lower()

train["text"]=train["text"].apply(lambda x :''.join(i for i in x if not i.isdigit()))
test["text"]=test["text"].apply(lambda x :''.join(i for i in x if not i.isdigit()))

test["text"]=test["text"].apply(lambda x: " ".join(filter(lambda y:(y[0]!='@') & (y[0]!=',@')&(y[0]!='#') & (y[0]!=',')
                                                            , x.split())))

train["text"]=train["text"].apply(lambda x: " ".join(filter(lambda y:(y[0]!='@') & (y[0]!=',@')&(y[0]!='#') & (y[0]!=',')
                                                            , x.split()))) 

/var/folders/wg/sphq_0pn1cd55c6h8mt_d_900000gn/T/ipykernel_51644/4266635964.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  train['text'] = train['text'].str.replace('[^\w\s]','')
/var/folders/wg/sphq_0pn1cd55c6h8mt_d_900000gn/T/ipykernel_51644/4266635964.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  test['text'] = test['text'].str.replace('[^\w\s]','')


In [25]:
stop = stopwords.words('english')
test['text_without_stopwords'] = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
train['text_without_stopwords'] = train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [26]:
test['tokenized_text'] = test.apply(lambda row: nltk.word_tokenize(row['text_without_stopwords']), axis=1)
train['tokenized_text'] = train.apply(lambda row: nltk.word_tokenize(row['text_without_stopwords']), axis=1)

In [27]:
train

,id,text,text_without_stopwords,tokenized_text
0,1016267329137729536,fhumumakhwanya hi no you cannot withdraw funds...,fhumumakhwanya hi cannot withdraw funds transf...,"[fhumumakhwanya, hi, can, not, withdraw, funds..."
1,1016270646060384256,gtalevi hi can you please share more informati...,gtalevi hi please share information regarding,"[gtalevi, hi, please, share, information, rega..."
2,1016272104839434240,mandilele hi thank you for sharing your experi...,mandilele hi thank sharing experience us,"[mandilele, hi, thank, sharing, experience, us]"
3,1016275249753743360,gtalevi please dm us your homeloan account num...,gtalevi please dm us homeloan account number c...,"[gtalevi, please, dm, us, homeloan, account, n..."
4,1016277410545889281,angvanreenen please provide us with it,angvanreenen please provide us,"[angvanreenen, please, provide, us]"
...,...,...,...,...
96557,1030149995490697217,dopla hi apologies for the delayed response pl...,dopla hi apologies delayed response please ref...,"[dopla, hi, apologies, delayed, response, plea..."
96558,1030162359933906945,blackroseza hi kindly dm your query affected p...,blackroseza hi kindly dm query affected phone ...,"[blackroseza, hi, kindly, dm, query, affected,..."
96559,1030170340473929730,aminahbhengu hi please inbox us your id and or...,aminahbhengu hi please inbox us id order numbe...,"[aminahbhengu, hi, please, inbox, us, id, orde..."
96560,1030172343937716224,reitumakwea hi please follow and inbox us your...,reitumakwea hi please follow inbox us query af...,"[reitumakwea, hi, please, follow, inbox, us, q..."


In [28]:
test

,id,text,text_without_stopwords,tokenized_text
0,1028546931453243392,warith hi wari please contact our customer com...,warith hi wari please contact customer complai...,"[warith, hi, wari, please, contact, customer, ..."
1,1028548116297986050,ziyaadseedat hi there ziyaad the samsung pay d...,ziyaadseedat hi ziyaad samsung pay also work v...,"[ziyaadseedat, hi, ziyaad, samsung, pay, also,..."
2,1028550157699350528,degreescc lovely packaging how long have you b...,degreescc lovely packaging long catering confe...,"[degreescc, lovely, packaging, long, catering,..."
3,1028552399307399169,consolerm hi there please give us your contact...,consolerm hi please give us contact details as...,"[consolerm, hi, please, give, us, contact, det..."
4,1028559460745150465,zimcritic loving yourself looks beautiful on y...,zimcritic loving looks beautiful tell us makes...,"[zimcritic, loving, looks, beautiful, tell, us..."
...,...,...,...,...
32205,1032883399898423298,leemasilo hi leemasilo sorry for late response...,leemasilo hi leemasilo sorry late response ple...,"[leemasilo, hi, leemasilo, sorry, late, respon..."
32206,1032886868902334464,beckechris thanks chris network informed netwo...,beckechris thanks chris network informed netwo...,"[beckechris, thanks, chris, network, informed,..."
32207,1032887467526049792,ntandoznmz youre welcome,ntandoznmz youre welcome,"[ntandoznmz, youre, welcome]"
32208,1032888433788825601,snemthethwa hi snemthethwa we would like to as...,snemthethwa hi snemthethwa would like assist m...,"[snemthethwa, hi, snemthethwa, would, like, as..."
